<a href="https://colab.research.google.com/github/NetoScientist/SqlServer/blob/main/DataScience%5CMinisterio_da_Saude_DB_SQLITE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando Biblioteca


In [68]:
import pandas as pd
from pandas import json_normalize
import requests
from sqlalchemy import create_engine

###Fazendo leitura via JSON

[Documentação](https://opendatasus.saude.gov.br/dataset/b772ee55-07cd-44d8-958f-b12edd004e0b/resource/5916b3a4-81e7-4ad5-adb6-b884ff198dc1/download/manual_api_vacina_covid-19.pdf)

URL para consumo da API Para consumir os dados disponibilizados pela API, utilizaremos a seguinte URL: https://imunizacao-es.saude.gov.br/_search Com a seguinte credencial de acesso:Usuário: imunizacao_public Senha: qlto5t&7r_@+#Tlstigi


In [3]:
r = requests.get('https://imunizacao-es.saude.gov.br/_search', 
                 auth=('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'))

In [4]:
json_content = r.json()

Verifcando a tabela

In [6]:
df = pd.DataFrame(json_content)
df

,took,timed_out,_shards,hits
total,1769,False,3.0,"{'value': 10000, 'relation': 'gte'}"
successful,1769,False,3.0,NaN
skipped,1769,False,0.0,NaN
failed,1769,False,0.0,NaN
max_score,1769,False,NaN,1
hits,1769,False,NaN,"[{'_index': 'desc-imunizacao', '_type': '_doc'..."


Testando a estrutura

In [7]:
json_content['hits']['hits'][0]

{'_id': '0eebbacc-1fa8-4447-b839-010e714f512b-i0b0',
 '_index': 'desc-imunizacao',
 '_score': 1.0,
 '_source': {'@timestamp': '2021-04-22T20:51:12.669Z',
  '@version': '1',
  'data_importacao_rnds': '2021-04-17 13:03:41',
  'document_id': '0eebbacc-1fa8-4447-b839-010e714f512b-i0b0',
  'estabelecimento_municipio_codigo': '291080',
  'estabelecimento_municipio_nome': 'FEIRA DE SANTANA',
  'estabelecimento_razaoSocial': 'PREFEITURA MUNICIPAL DE FEIRA DE SANTANA',
  'estabelecimento_uf': 'BA',
  'estabelecimento_valor': '7430671',
  'estalecimento_noFantasia': 'REDE FRIO',
  'id_sistema_origem': '17958',
  'paciente_dataNascimento': '1959-07-02',
  'paciente_endereco_cep': '44068',
  'paciente_endereco_coIbgeMunicipio': '291080',
  'paciente_endereco_coPais': '10',
  'paciente_endereco_nmMunicipio': 'FEIRA DE SANTANA',
  'paciente_endereco_nmPais': 'BRASIL',
  'paciente_endereco_uf': 'BA',
  'paciente_enumSexoBiologico': 'M',
  'paciente_id': '03fcfb7ca3b42888eb61871960b9a9af74b1154844afd2

Testando com 10 K

In [11]:
r_scroll = requests.post('https://imunizacao-es.saude.gov.br/_search?scroll=1m', 
                 auth=('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'), 
                 json={'size': 10000})

In [12]:
scroll = r_scroll.json()

Verificando as colunas

In [46]:
cols = scroll['hits']['hits'][0]['_source'].keys()

In [47]:
cols

dict_keys(['paciente_id', 'document_id', 'paciente_endereco_coIbgeMunicipio', 'vacina_categoria_codigo', '@timestamp', 'estabelecimento_razaoSocial', 'vacina_codigo', 'estabelecimento_municipio_codigo', 'paciente_nacionalidade_enumNacionalidade', 'vacina_categoria_nome', 'vacina_grupoAtendimento_codigo', 'paciente_endereco_nmPais', 'vacina_dataAplicacao', 'estabelecimento_municipio_nome', 'paciente_endereco_coPais', 'vacina_lote', 'sistema_origem', 'vacina_nome', '@version', 'paciente_endereco_uf', 'estalecimento_noFantasia', 'vacina_grupoAtendimento_nome', 'vacina_fabricante_nome', 'estabelecimento_uf', 'paciente_racaCor_codigo', 'vacina_descricao_dose', 'paciente_racaCor_valor', 'paciente_enumSexoBiologico', 'data_importacao_rnds', 'id_sistema_origem', 'paciente_dataNascimento', 'estabelecimento_valor', 'paciente_endereco_cep', 'paciente_idade', 'paciente_endereco_nmMunicipio'])

Criando Series PANDAS

In [62]:
source = []
for item in scroll['hits']['hits']:
  source.append(pd.Series(item['_source']))


In [66]:
df_hits = pd.DataFrame(source)
df_hits.head()

,paciente_id,document_id,paciente_endereco_coIbgeMunicipio,vacina_categoria_codigo,@timestamp,estabelecimento_razaoSocial,vacina_codigo,estabelecimento_municipio_codigo,paciente_nacionalidade_enumNacionalidade,vacina_categoria_nome,vacina_grupoAtendimento_codigo,paciente_endereco_nmPais,vacina_dataAplicacao,estabelecimento_municipio_nome,paciente_endereco_coPais,vacina_lote,sistema_origem,vacina_nome,@version,paciente_endereco_uf,estalecimento_noFantasia,vacina_grupoAtendimento_nome,vacina_fabricante_nome,estabelecimento_uf,paciente_racaCor_codigo,vacina_descricao_dose,paciente_racaCor_valor,paciente_enumSexoBiologico,data_importacao_rnds,id_sistema_origem,paciente_dataNascimento,estabelecimento_valor,paciente_endereco_cep,paciente_idade,paciente_endereco_nmMunicipio,vacina_fabricante_referencia
0,1d6a071be89105915e8356984b12095912d4c132414a56...,04219b39-7515-496b-abb2-2cb3f2d36bab-i0b0,352930,2,2021-04-22T20:39:14.448Z,SOCIEDADE MATONENSE DE BENEMERENCIA,86,352930,E,Faixa Etária,000201,BRASIL,2021-01-22T00:00:00.000Z,MATAO,10,202010033,VACIVIDA,Covid-19-Coronavac-Sinovac/Butantan,1,SP,HOSPITAL CARLOS FERNANDO MALZONI MATAO,Pessoas de 60 a 64 anos,SINOVAC,SP,01,1ª Dose,BRANCA,M,2021-01-22 03:01:00,18262,1959-10-05,2090961,15997,61,MATAO,NaN
1,6863519797fe60a5b19623ce23bfa3f08044388363a0c6...,f99e6248-8188-4ca3-9307-2e2a700b11e9-i0b0,353470,2,2021-04-22T20:39:14.800Z,PREFEITURA MUNICIPAL DE OURINHOS,86,353470,B,Faixa Etária,000204,BRASIL,2021-04-05T00:00:00.000Z,OURINHOS,10,210091,VACIVIDA,Covid-19-Coronavac-Sinovac/Butantan,1,SP,UBS VILA MARGARIDA,Pessoas de 75 a 79 anos,SINOVAC,SP,01,2ª Dose,BRANCA,F,2021-04-05 03:00:00,18262,1944-07-24,2046482,19907,76,OURINHOS,NaN
2,f9666ceab4de760ea690ae23ad6380ec6a28aa071bfa1d...,79e0e12e-3208-4e70-8d29-a6a509448fba-i0b0,240450,9,2021-04-22T20:39:15.011Z,PREFEITURA MUNICIPAL DE GUAMARE,86,240450,B,Trabalhadores de Saúde,000923,BRASIL,2021-01-21T00:00:00.000Z,GUAMARE,10,202010028,RN + Vacina,Covid-19-Coronavac-Sinovac/Butantan,1,RN,UNIDADE BASICA DE GUAMARE,Técnico de Enfermagem,FUNDACAO BUTANTAN,RN,99,1ª Dose,SEM INFORMACAO,F,2021-03-19 01:54:28,17862,1968-12-05,3686019,59598,52,GUAMARE,NaN
3,1513cc0ec3354a554a6e0065617110d153406e33a51429...,bf980662-e1a2-46b3-bdd8-2ab4d5b8106d-i0b0,320500,9,2021-04-22T20:39:15.042Z,PREFEITURA MUNICIPAL DA SERRA,85,320500,B,Trabalhadores de Saúde,000926,BRASIL,2021-01-29T03:00:00.000Z,SERRA,10,4120Z005,Novo PNI,Vacina Covid-19 - Covishield,1,ES,UNIDADE REGIONAL DE SAUDE PEDRO FEU ROSA,Outros,FUNDACAO OSWALDO CRUZ,ES,03,1ª Dose,PARDA,F,2021-02-04 12:03:23,16341,1968-10-31,2499436,29165,52,SERRA,Organization/33781055000135
4,6bd0f6e65ce6d0c08099fa547ecfe52e5dfcb454653d91...,d8942cc6-bd3b-495e-9602-778fffc96504-i0b0,353800,9,2021-04-22T20:39:15.241Z,MUNICIPIO DE PINDAMONHANGABA,86,353800,B,Trabalhadores de Saúde,000926,BRASIL,2021-01-22T00:00:00.000Z,PINDAMONHANGABA,10,202010033,VACIVIDA,Covid-19-Coronavac-Sinovac/Butantan,1,SP,UNIDADE BASICA DE SAUDE CENTRO PINDAMONHANGABA,Outros,SINOVAC,SP,01,1ª Dose,BRANCA,M,2021-01-22 17:01:25,18262,1981-04-06,2755068,12420,39,PINDAMONHANGABA,NaN


In [67]:
save_df = df_hits

In [69]:
engine = create_engine('sqlite:///save_dfpandas.db', echo=True)
sqlite_connection = engine.connect()

In [70]:
sqlite_table = "API_SAUDE"
save_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-05-03 18:50:02,053 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("API_SAUDE")
2021-05-03 18:50:02,055 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-05-03 18:50:02,059 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("API_SAUDE")
2021-05-03 18:50:02,060 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-05-03 18:50:02,074 INFO sqlalchemy.engine.Engine 
CREATE TABLE "API_SAUDE" (
	"index" BIGINT, 
	paciente_id TEXT, 
	document_id TEXT, 
	"paciente_endereco_coIbgeMunicipio" TEXT, 
	vacina_categoria_codigo TEXT, 
	"@timestamp" TEXT, 
	"estabelecimento_razaoSocial" TEXT, 
	vacina_codigo TEXT, 
	estabelecimento_municipio_codigo TEXT, 
	"paciente_nacionalidade_enumNacionalidade" TEXT, 
	vacina_categoria_nome TEXT, 
	"vacina_grupoAtendimento_codigo" TEXT, 
	"paciente_endereco_nmPais" TEXT, 
	"vacina_dataAplicacao" TEXT, 
	estabelecimento_municipio_nome TEXT, 
	"paciente_endereco_coPais" TEXT, 
	vacina_lote TEXT, 
	sistema_origem TEXT, 
	vacina_nome TEXT, 
	"@version" TEXT, 
	pac

In [71]:
sqlite_connection.close()